In [2]:
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator
from langchain_core.runnables import ConfigurableField,ConfigurableFieldSpec
from langchain_community.chat_message_histories import ChatMessageHistory,RedisChatMessageHistory  # new
from langchain_core.chat_history import BaseChatMessageHistory  # new
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory  # new

In [3]:
load_dotenv(find_dotenv("../.env"))

True

In [4]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmGPT=ChatOpenAI(model="gpt-4o-mini")

In [5]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            template="You're an assistant who's good at {ability}. Respond in 20 words or fewer"
        ),
        MessagesPlaceholder(
            variable_name="history"
        ),
        HumanMessagePromptTemplate.from_template(
            template="{question}"
        )
    ]
)

In [6]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ability'], input_types={}, partial_variables={}, template="You're an assistant who's good at {ability}. Respond in 20 words or fewer"), additional_kwargs={}),
 MessagesPlaceholder(variable_name='history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]

In [7]:
chain=prompt|llmGPT

<h3>In Memory Chat Message History </h3>

In [21]:
store={}
def getSessionHistory(sessionId:str) -> BaseChatMessageHistory:
    if sessionId not in store:
        store[sessionId]=ChatMessageHistory()
    return store[sessionId]

In [22]:
isinstance(ChatMessageHistory(),BaseChatMessageHistory)

True

In [23]:
chainWithMessageHistory=RunnableWithMessageHistory(
                            runnable=chain,
                            get_session_history=getSessionHistory,
                            input_messages_key="question",
                            history_messages_key="history"
                        )

In [24]:
sessionID="ritish"

In [25]:
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"Which is the smallest country in the world?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='The smallest country in the world is Vatican City, covering about 44 hectares (110 acres).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 35, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cxni7mCbalJ8yj61DqjDvYxcL1q6M', 'finish_reason': 'stop', 'logprobs': None}, id='run--45dd0b7e-f66b-49a8-82b1-cd91d9e41dc7-0', usage_metadata={'input_tokens': 35, 'output_tokens': 19, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [26]:
# Remembers
chainWithMessageHistory.invoke(
    input={"ability":"Geography",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='Vatican City is located within the city of Rome, Italy.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 67, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cxni8emKBYCsFtJ3ggDylmmApNKlC', 'finish_reason': 'stop', 'logprobs': None}, id='run--ba97e9b3-da02-432a-b422-0f15d93db7ee-0', usage_metadata={'input_tokens': 67, 'output_tokens': 14, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [27]:
store

{'ritish': InMemoryChatMessageHistory(messages=[HumanMessage(content='Which is the smallest country in the world?', additional_kwargs={}, response_metadata={}), AIMessage(content='The smallest country in the world is Vatican City, covering about 44 hectares (110 acres).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 35, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cxni7mCbalJ8yj61DqjDvYxcL1q6M', 'finish_reason': 'stop', 'logprobs': None}, id='run--45dd0b7e-f66b-49a8-82b1-cd91d9e41dc7-0', usage_metadata={'input_tokens': 35, 'output_tokens': 19, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio

In [29]:
store[sessionID]

InMemoryChatMessageHistory(messages=[HumanMessage(content='Which is the smallest country in the world?', additional_kwargs={}, response_metadata={}), AIMessage(content='The smallest country in the world is Vatican City, covering about 44 hectares (110 acres).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 35, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cxni7mCbalJ8yj61DqjDvYxcL1q6M', 'finish_reason': 'stop', 'logprobs': None}, id='run--45dd0b7e-f66b-49a8-82b1-cd91d9e41dc7-0', usage_metadata={'input_tokens': 35, 'output_tokens': 19, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reas

In [30]:
store[sessionID].messages

[HumanMessage(content='Which is the smallest country in the world?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The smallest country in the world is Vatican City, covering about 44 hectares (110 acres).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 35, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cxni7mCbalJ8yj61DqjDvYxcL1q6M', 'finish_reason': 'stop', 'logprobs': None}, id='run--45dd0b7e-f66b-49a8-82b1-cd91d9e41dc7-0', usage_metadata={'input_tokens': 35, 'output_tokens': 19, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content

In [31]:
# Will not remember due to different sessionID
chainWithMessageHistory.invoke(
    input={"ability":"What?",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":"Tania"}}
    )

AIMessage(content='Please specify what you are referring to for location details.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 31, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CxniPuvLvNfExRlpiTwvByNwXVUkh', 'finish_reason': 'stop', 'logprobs': None}, id='run--181fe9d2-b0fd-4303-8a51-651e282955bb-0', usage_metadata={'input_tokens': 31, 'output_tokens': 11, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

<h3> Using Configurable Field Spec to include two parameters: userID and conversationID </h3> 

In [32]:
store={}
def getSessionHistory(userID:str, conversationID:str) -> BaseChatMessageHistory:
    if (userID,conversationID) not in store:
        store[(userID,conversationID)]=ChatMessageHistory()
    return store[(userID,conversationID)]

In [33]:
chainWithMessageHistory=RunnableWithMessageHistory(
                            runnable=chain,
                            get_session_history=getSessionHistory,
                            input_messages_key="question",
                            history_messages_key="history",
                            history_factory_config=[
                                ConfigurableFieldSpec(
                                    id="userID",
                                    annotation=str,
                                    name="User ID",
                                    description="Unique Identifier for the User",
                                    default="",
                                    is_shared=True
                                    ),
                                ConfigurableFieldSpec(
                                    id="conversationID",
                                    annotation=str,
                                    name="Conversation ID",
                                    description="Unique Identifier for the Conversation",
                                    default="",
                                    is_shared=True
                                    )
                            ]
                        )

In [36]:
userID="ritish"
conversationID="1"

In [37]:
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"Which is the smallest country in the world?"},
    config={"configurable":{"userID":userID,"conversationID":conversationID}}
    )

AIMessage(content='Vatican City is the smallest country in the world, located entirely within Rome, Italy.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 64, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cxnjn3Bts6CkdHRChhTDDQpbwDSbJ', 'finish_reason': 'stop', 'logprobs': None}, id='run--d6be7951-f5b2-4747-b126-069e43afd75b-0', usage_metadata={'input_tokens': 64, 'output_tokens': 19, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [35]:
# Remembers
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"Where is it located?"},
    config={"configurable":{"userID":userID,"conversationID":conversationID}}
    )

AIMessage(content='Please specify the location you are inquiring about so I can provide accurate information.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 31, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CxnjeXVE0RfahEHzjpXx4O1qdSbRt', 'finish_reason': 'stop', 'logprobs': None}, id='run--cfa8f85a-2efd-432d-bc77-806c0b09c4be-0', usage_metadata={'input_tokens': 31, 'output_tokens': 16, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [38]:
# Will not remember due to different conversationID
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"What is it's area?"},
    config={"configurable":{"userID":userID,"conversationID":"2"}}
    )

AIMessage(content='Please specify which location or entity you are referring to for its area information.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 31, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CxnkCWsscEuRpzn4aCFAPWy2lowL2', 'finish_reason': 'stop', 'logprobs': None}, id='run--15a85337-7aee-43e6-bdc7-3d813a03a9a5-0', usage_metadata={'input_tokens': 31, 'output_tokens': 15, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [40]:
store[(userID,conversationID)].messages

[HumanMessage(content='Where is it located?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Please specify the location you are inquiring about so I can provide accurate information.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 31, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CxnjeXVE0RfahEHzjpXx4O1qdSbRt', 'finish_reason': 'stop', 'logprobs': None}, id='run--cfa8f85a-2efd-432d-bc77-806c0b09c4be-0', usage_metadata={'input_tokens': 31, 'output_tokens': 16, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='Which is the smallest

<h3>Use with Redis</h3>

In [24]:
REDIS_URL="redis://localhost:6379/0"

In [25]:
def getMessageHistory(sessionID:str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id=sessionID,url=REDIS_URL)

In [26]:
chainWithMessageHistoryRedis=RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=getMessageHistory,
    input_messages_key="question",
    history_messages_key="history"
)

In [28]:
chainWithMessageHistoryRedis.invoke(
    input={"ability":"geography",
           "question":"Which is the smallest country in the world?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='Vatican City, an independent city-state enclaved within Rome, Italy, is the smallest country in the world.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-c08601d0-388e-4c21-89a0-ff49459e8451-0', usage_metadata={'input_tokens': 29, 'output_tokens': 26, 'total_tokens': 55, 'input_token_details': {'cache_read': 0}})

In [29]:
# Remembers
chainWithMessageHistoryRedis.invoke(
    input={"ability":"geography",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='Vatican City is located within Rome, Italy.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-62b382ee-87ea-4a43-a22c-85d45c1a0cdc-0', usage_metadata={'input_tokens': 59, 'output_tokens': 11, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}})

In [30]:
# Will not remember due to different sessionID
chainWithMessageHistoryRedis.invoke(
    input={"ability":"geography?",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":"Tania"}}
    )

AIMessage(content='To answer that, I need to know what "it" refers to! Please provide more context. ', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-0b92f900-e43a-4f9e-97a8-d80942f44a14-0', usage_metadata={'input_tokens': 25, 'output_tokens': 22, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}})